In [8]:
!pip cache purge
!pip install --upgrade torch torchvision


Files removed: 146
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 818.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [5]:
!pip install -q openai python-dotenv transformers torch tqdm accelerate

In [1]:
# from dotenv import load_dotenv
import os

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
# userdata.get('OPENAI_API_KEY')

# load_dotenv()
# huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
# openai_token = os.getenv("OPENAI_API_KEY")

In [2]:
from google.colab import drive
import os
import zipfile
from tqdm import tqdm
# from dotenv import load_dotenv
from openai import OpenAI
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
# ========== Mount Google Drive ==========
drive.mount('/content/drive')

# ========== Path Configuration ==========
# Update these paths according to your Google Drive structure
DRIVE_BASE = '/content/drive/MyDrive/'
ZIP_PATH = "./data/elmundo_chunked_es_page1_40years.zip"
EXTRACT_DIR = '/tmp/extracted'  # Using tmp for faster I/O
OUTPUT_DIR = os.path.join(DRIVE_BASE, 'cleaned_articles1')

# Create directories
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


Mounted at /content/drive


In [3]:
ZIP_PATH = "./data/elmundo_chunked_es_page1_40years.zip"
EXTRACT_DIR = '/tmp/extracted'  # Using tmp for faster I/O
os.makedirs(EXTRACT_DIR, exist_ok=True)

In [4]:
# ========== File Extraction ==========
def extract_files():
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        # Extract nested structure
        for file in zip_ref.namelist():
            if file.endswith('.txt'):
                zip_ref.extract(file, EXTRACT_DIR)
                print(f"{file} extracted")

    print(f"Extracted files to: {EXTRACT_DIR}")


In [10]:
# ========== OpenAI Correction ==========
def correct_with_openai(text):
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """Eres un experto en documentos históricos de Puerto Rico. El texto en español son noticias del siglo XX y contiene muchos errores a causa del OCR. Descifra el contenido y tradúcelo al inglés:
1. Preserva nombres propios (ej: Mayagüez, Caguas)
2. Ignora el "header" (ej:
```EL MUNDO
PRONOSTICOS DEL TIEMPO PARA LA ISLA, HOY: Mayormente nublado, con aguaceros dispersos temprano en la mafiana. EN SAN JUAN. AYER: Temperatura máxima. 80; mínima, 77. Presión barométrica al nivel del mar, a las 4:80 de la tarde. 38.88 pulgadas de mercurio. No hay indicios de disturbio tropical.
40 páginas 5/
DIARIO DE LA MARANA
AÑO XXVIII
Entered aa second clsss matter, Post Office, San Juan, P. R.)```
3. Ignora los anuncios
4. Solo mantén contenido relacionado a Puerto Rico (especialmente sobre ciudades, locaciones o eventos históricos)
5. Traduce el texto a inglés. Solo mantén los datos mas importantes
6. Lista las ciudades o locaciones de Puerto Rico mencionadas
7. Escribe solo en texto (no uses **negrillas** ni *itálicas* ni nada en markdown)
8. return it as a JSON object with two fields:
    - 'metadata': un diccionario con la siguiente informacion: 'file_name' (nombre del archivo), 'date' (fecha del articulo), 'locations' (lista de las ciudades o locaciones de Puerto Rico mencionadas).
    - 'text': the corrected and summarized text in English.
8. No digas nada mas ni preguntes más. Usa el siguiente texto:"""
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=0.1,
        max_tokens=2000
    )
    return response.choices[0].message.content

In [8]:
# !pip install -q transformers==4.38.2 torch==2.2.1
# !rm -rf ~/.cache/huggingface/hub  # Clear corrupted cache

# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import requests

# # ========== Verify Internet Access ==========
# try:
#     response = requests.get("https://huggingface.co/google/flan-t5-small", timeout=10)
#     print("Hugging Face Hub is reachable ✅")
# except:
#     print("NO INTERNET ACCESS TO HUGGINGFACE ❌")
#     # If using Colab: Runtime → Disconnect & delete runtime → Start fresh


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


try:
    tokenizer = T5Tokenizer.from_pretrained(
        "google/flan-t5-small",
        force_download=True,  # Bypass cache
        resume_download=False
    )
    model = T5ForConditionalGeneration.from_pretrained(
        "google/flan-t5-small",
        device_map="auto"
    )
    print("Model loaded successfully! 🎉")
except Exception as e:
    print(f"Error: {str(e)}")
    # Fallback to local copy if available

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Error: Force download failed due to the above error.


In [6]:
!mkdir -p flan-t5-small
%cd flan-t5-small

# Download all required files manually
!wget -q https://huggingface.co/google/flan-t5-small/resolve/main/config.json
!wget -q https://huggingface.co/google/flan-t5-small/resolve/main/pytorch_model.bin
!wget -q https://huggingface.co/google/flan-t5-small/resolve/main/special_tokens_map.json
!wget -q https://huggingface.co/google/flan-t5-small/resolve/main/spiece.model
!wget -q https://huggingface.co/google/flan-t5-small/resolve/main/tokenizer_config.json
!wget -q https://huggingface.co/google/flan-t5-small/resolve/main/generation_config.json

%cd ..

from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load from local directory
tokenizer = T5Tokenizer.from_pretrained("./flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("./flan-t5-small", device_map="auto")


def correct_with_opensource(text):
    inputs = tokenizer(
        f"Este texto es de un periodico llamado 'El Mundo' y contiene noticias de Puerto Rico del siglo XX. Corrige errores OCR en este texto español manteniendo nombres propios y formato: {text}",
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(model.device)

    outputs = model.generate(
        inputs.input_ids,
        max_length=1024,
        num_beams=3
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

/content/flan-t5-small


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/content


In [8]:
# ========== Summarize Text ==========
# Load model directly

# !mkdir -p bert2bert-shared-spanish-summarization
# %cd bert2bert-shared-spanish-summarization

# # Download all required files manually
# !wget -q https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization/resolve/main/config.json
# !wget -q https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization/resolve/main/pytorch_model.bin
# !wget -q https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization/resolve/main/special_tokens_map.json
# !wget -q https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization/resolve/main/tokenizer_config.json
# !wget -q https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization/resolve/main/vocab.txt

# %cd ..

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load from local directory
sum_tokenizer = AutoTokenizer.from_pretrained("./bert2bert-shared-spanish-summarization")
sum_model = AutoModelForSeq2SeqLM.from_pretrained("./bert2bert-shared-spanish-summarization")#, device_map="auto")

def summarize_text(text):
    inputs = sum_tokenizer(
        text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to("cuda")

    outputs = sum_model.generate(
        inputs.input_ids,
        max_length=512,
        min_length=256,
        num_beams=4,
        early_stopping=True
    )
    return sum_tokenizer.decode(outputs[0], skip_special_tokens=True)

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "dccuchile/bert-base-span

In [10]:
!mkdir -p opus-mt-es-en
%cd opus-mt-es-en
!wget -q https://huggingface.co/Helsinki-NLP/opus-mt-es-en/resolve/main/config.json
!wget -q https://huggingface.co/Helsinki-NLP/opus-mt-es-en/resolve/main/pytorch_model.bin
!wget -q https://huggingface.co/Helsinki-NLP/opus-mt-es-en/resolve/main/tokenizer_config.json
!wget -q https://huggingface.co/Helsinki-NLP/opus-mt-es-en/resolve/main/source.spm
!wget -q https://huggingface.co/Helsinki-NLP/opus-mt-es-en/resolve/main/target.spm
%cd ..


/content/opus-mt-es-en
/content


In [18]:
!wget -q https://huggingface.co/Helsinki-NLP/opus-mt-es-en/resolve/main/tokenizer.json -P ./opus-mt-es-en



In [19]:
# ========== Translation Pipeline ==========
# translator = pipeline("translation_es_to_en",
#                      model="Helsinki-NLP/opus-mt-es-en",
#                      device=0)

from transformers import MarianMTModel, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained(
    "./opus-mt-es-en",
    source_spm="./opus-mt-es-en/source.spm",
    target_spm="./opus-mt-es-en/target.spm",

)
model = AutoModelForSeq2SeqLM.from_pretrained("./opus-mt-es-en")

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Translation pipeline
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [23]:
from sentence_splitter import SentenceSplitter

# ========== Chunk text ==========
splitter = SentenceSplitter(language='es')
def chunk_text(text, max_chars=1000):
    """Split text into meaningful chunks preserving sentence boundaries"""
    chunks = []
    current_chunk = []
    current_len = 0

    for sentence in splitter.split(text):
        sent_len = len(sentence)
        if current_len + sent_len > max_chars:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_len = sent_len
        else:
            current_chunk.append(sentence)
            current_len += sent_len

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks


In [39]:
# ========== Processing Pipeline ==========
def process_files():
    extract_files()

    # Get all text files from nested directory
    base_dir = os.path.join(EXTRACT_DIR, "elmundo_chunked_es_page1_40years")
    txt_files = [f for f in os.listdir(base_dir) if f.endswith('.txt')]

    for filename in tqdm(txt_files, desc="Processing files"):
        input_path = os.path.join(base_dir, filename)
        output_path = os.path.join(OUTPUT_DIR, f"cleaned_{filename}")

        with open(input_path, 'r', encoding='utf-8', errors='ignore') as f:
            raw_text = f.read()

        try:
            # Choose one method:
            cleaned_text = correct_with_openai(raw_text)  # OpenAI version
            # cleaned_text = [correct_with_opensource(chunk) for chunk in chunk_text(raw_text)]  # Open-source version

            # if len(cleaned_text.split()) > 750:  # Only summarize long articles
            #     summary = summarize_text(cleaned_text)
            # else:
            #     summary = cleaned_text

            # # 3. Translate
            # translated = translator(summary, max_length=600)[0]['translation_text']

            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(cleaned_text)

            print(f"Processed: {filename} -> Saved to Drive")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            continue

In [24]:
input_path = '19470802_1.txt'

with open(input_path, 'r', encoding='utf-8', errors='ignore') as f:
            raw_text = f.read()

cleaned_text = [correct_with_opensource(chunk) for chunk in chunk_text(raw_text)]  # Open-source version

# if len(cleaned_text.split()) > 750:  # Only summarize long articles
#     summary = summarize_text(cleaned_text)
# else:
#     summary = cleaned_text

# # 3. Translate
# translated = translator(summary, max_length=600)[0]['translation_text']

print(cleaned_text)


KeyboardInterrupt: 

In [40]:
process_files()
print(f"\nAll cleaned files saved to: {OUTPUT_DIR}")

elmundo_chunked_es_page1_40years/19200103_1.txt extracted
elmundo_chunked_es_page1_40years/19200110_1.txt extracted
elmundo_chunked_es_page1_40years/19200117_1.txt extracted
elmundo_chunked_es_page1_40years/19200124_1.txt extracted
elmundo_chunked_es_page1_40years/19200131_1.txt extracted
elmundo_chunked_es_page1_40years/19200207_1.txt extracted
elmundo_chunked_es_page1_40years/19200214_1.txt extracted
elmundo_chunked_es_page1_40years/19200221_1.txt extracted
elmundo_chunked_es_page1_40years/19200228_1.txt extracted
elmundo_chunked_es_page1_40years/19200306_1.txt extracted
elmundo_chunked_es_page1_40years/19200313_1.txt extracted
elmundo_chunked_es_page1_40years/19200327_1.txt extracted
elmundo_chunked_es_page1_40years/19200403_1.txt extracted
elmundo_chunked_es_page1_40years/19200410_1.txt extracted
elmundo_chunked_es_page1_40years/19200417_1.txt extracted
elmundo_chunked_es_page1_40years/19200424_1.txt extracted
elmundo_chunked_es_page1_40years/19200501_1.txt extracted
elmundo_chunke

Processing files:   0%|          | 1/1668 [00:04<2:18:33,  4.99s/it]

Processed: 19510414_1.txt -> Saved to Drive


Processing files:   0%|          | 2/1668 [00:05<1:07:23,  2.43s/it]

Processed: 19450407_1.txt -> Saved to Drive


...

Processed: 19510203_1.txt -> Saved to Drive


Processing files:   8%|▊         | 129/1668 [18:03<6:09:27, 14.40s/it]

Processed: 19351123_1.txt -> Saved to Drive


Processing files:   8%|▊         | 129/1668 [18:04<3:35:41,  8.41s/it]


KeyboardInterrupt: 